In [ ]:
pip install pandas

In [ ]:
import torch
import pandas as pd
import torch.nn as nn
import matplotlib.pyplot as plt
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import numpy as np

In [ ]:
# Load the training and testing datasets
file_path = r"C:\Users\Asus\Desktop\X_test.csv" 
X_test = pd.read_csv(file_path)  
X_test = np.array(X_test)
new_array = X_test[:, 1:]
new = torch.from_numpy(new_array)
print(new)

file_path_tr = r"C:\Users\Asus\Desktop\X_train.csv" 
X_train = pd.read_csv(file_path_tr)    

# Remove nule rows
X_train = X_train[~(X_train[X_train.columns[1:-1]] == 0).all(axis=1)]
novo_X_train = X_train[X_train['t'] == 0]

#
X_treino, X_valid = train_test_split(novo_X_train, test_size=0.2)
X_treino = np.array(X_treino)
X_train = np.array(X_train)
Y = X_train[:,[1,2,5,6,9,10]]

Y = X_train[:,[1,2,5,6,9,10]]
X = torch.tensor(X_treino[:, [0,1,2,5,6,9,10]], dtype=torch.float32)
X.shape


In [ ]:

Y = torch.tensor(Y, dtype=torch.float32)
model = nn.Sequential(
    nn.Linear(7, 64).to(torch.float32),
    nn.ReLU(),
    nn.Linear(64, 128).to(torch.float32),
    nn.BatchNorm1d(128),
    nn.ReLU(),
    nn.Linear(128, 256).to(torch.float32),
    nn.BatchNorm1d(256),
    nn.ReLU(),
    nn.Linear(256, 128).to(torch.float32),
    nn.BatchNorm1d(128),
    nn.ReLU(),
    nn.Linear(128, 64).to(torch.float32),
    nn.BatchNorm1d(64),
    nn.ReLU(),
    nn.Linear(64, 32).to(torch.float32),
    nn.BatchNorm1d(32),
    nn.ReLU(),
    nn.Linear(32, 6).to(torch.float32))
model

loss_fn   = nn.SmoothL1Loss() 
optimizer = optim.Adam(model.parameters(), lr=0.001)

n_epochs = 200
batch_size = 257

for epoch in range(n_epochs):
    for i in range(0, n_epochs, 1):
       Xbatch = np.tile(X[i], 257)
    
       Xbatch = torch.from_numpy(Xbatch)
       Xbatch = Xbatch.reshape(257, 7)
       for t in range(0, 257):
        Xbatch[t, 0] = t*(10/257)

       y_pred = model(Xbatch)
       ybatch = Y[i:i+batch_size]
       loss_value = loss_fn(y_pred, ybatch)
       optimizer.zero_grad()
       loss_value.backward()
       optimizer.step()
    print(f'Finished epoch {epoch}, latest loss {loss_value}')

In [ ]:
n_epochs = 4053
results = []
all_predictions = np.empty((0,6))


with torch.no_grad():  # Disable gradient tracking since it's inference
    model.eval()
    for i in range(n_epochs):
        newnew = new[(i*257):(i*257+257),:]
        newnew = newnew.to(torch.float32)
        y_pred_test = model(newnew)
        all_predictions = np.vstack((all_predictions, y_pred_test))


In [ ]:

all_predictions

In [ ]:
#Write the predictions file

# Crie a string de cabeçalho
header = "Id,x_1,y_1,x_2,y_2,x_3,y_3"

# Converta o tensor em uma lista de listas e formate os valores
formatted_data = []
for idx, row in enumerate(all_predictions):
    formatted_row = [f"{idx},{value:.3f}" for value in row]
    formatted_data.append(",".join(formatted_row))

# Combine os dados formatados em uma única string
formatted_output = "\n".join([header] + formatted_data)

In [ ]:
import csv

In [ ]:
# Abra um arquivo CSV para escrever
with open('output.csv', 'w', newline='') as csvfile:
    # Crie um escritor CSV
    csv_writer = csv.writer(csvfile)

    # Escreva o cabeçalho
    csv_writer.writerow(["Id", "x_1", "y_1", "x_2", "y_2", "x_3", "y_3"])

    # Escreva os dados formatados
    for idx, row in enumerate(all_predictions):
        formatted_row = [idx] + [round(val.item(), 3) for val in row]
        csv_writer.writerow(formatted_row)